In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil

import tensorflow as tf
import h5py

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.regularizers import l2
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.utils as utils

from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
import zipfile

In [2]:
drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/Colab Notebooks/car_damage_type_0703.zip'

re_path = '/content/car_damage_type_0703'

with zipfile.ZipFile(zip_path, 'r') as target_file:
    for file in target_file.namelist():
        file_path = os.path.join(re_path, file)
        if not os.path.exists(file_path):  # 이미 파일이 존재하는 경우 건너뛰기
            target_file.extract(file, re_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_data_dir = '/content/car_damage_type_0703/car_damage_type_0703/car_damage_type/train'
test_data_dir = '/content/car_damage_type_0703/car_damage_type_0703/car_damage_type/test'
image_size = (224, 224)
batch_size = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0,
                                                                rotation_range=45,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                zoom_range=0.2)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0,
                                                                rotation_range=45,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                zoom_range=0.2)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 11559 images belonging to 4 classes.
Found 4452 images belonging to 4 classes.


In [ ]:
# 모델 구성
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(image_size[0], image_size[1],3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
start_time = datetime.now()

result = model.fit(x=train_generator, epochs=20, validation_data=test_generator, callbacks=[early_stopping])

end_time = datetime.now()

print('걸린시간:', end_time - start_time)

# 모델 평가
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss : {test_loss}')
print(f'Test Accuracy : {test_accuracy}')

Epoch 1/20
557/557 [==============================] - 521s 912ms/step - loss: 1.4669 - accuracy: 0.2417 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 2/20
557/557 [==============================] - 505s 907ms/step - loss: 1.3864 - accuracy: 0.2489 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 3/20
557/557 [==============================] - 500s 898ms/step - loss: 1.3864 - accuracy: 0.2451 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 4/20
519/557 [==========================>...] - ETA: 27s - loss: 1.3864 - accuracy: 0.2502

In [ ]:
base_model = MobileNetV2(input_shape=(image_size[0], image_size[1], 3), include_top=False, weights='imagenet')

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

start_time = datetime.now()

result = model.fit(x=train_generator, epochs=30, validation_data=test_generator, callbacks=[early_stopping])

end_time = datetime.now()

print('걸린 시간:', end_time - start_time)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Epoch 1/30
362/362 [==============================] - 363s 888ms/step - loss: 3.7768 - accuracy: 0.3346 - val_loss: 3.9872 - val_accuracy: 0.3115
Epoch 2/30
362/362 [==============================] - 315s 870ms/step - loss: 3.5110 - accuracy: 0.4272 - val_loss: 3.8477 - val_accuracy: 0.3628
Epoch 3/30
362/362 [==============================] - 369s 1s/step - loss: 3.3671 - accuracy: 0.4936 - val_loss: 3.6507 - val_accuracy: 0.4023
Epoch 4/30
362/362 [==============================] - 312s 861ms/step - loss: 3.2568 - accuracy: 0.5258 - val_loss: 3.4917 - val_accuracy: 0.4438
Epoch 5/30
362/362 [==============================] - 310s 858ms/step - loss: 3.1467 - accuracy: 0.5651 - val_loss: 3.3778 - val_accuracy: 0.4695
Epoch 6/30
362/362 [==============================] - 312s 863ms/step - loss: 3.0530 - accuracy: 0.5917 - val_loss: 3.2678 - val_accuracy: 0.4975
Epoch 7/30
351/362 [============================>.] - ETA: 6s - loss: 2.9754 - accuracy: 0.6052

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)

plt.title('LOSS')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()

plt.plot(result.history['loss'], label='train_loss')
plt.plot(result.history['val_loss'], label= 'validation_loss')
plt.legend(loc='best')

plt.subplot(1,2,2)

plt.title('ACCURACY')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.grid()

plt.plot(result.history['accuracy'], label='train_accuracy')
plt.plot(result.history['val_accuracy'], label= 'validationi_accuracy')
plt.legend(loc='best')
